# TFT Simulation

In [1]:
from lib import *
simulation = j.tools.tfgrid_simulator.simulation_get(
            name="default",
            tokencreator_name="optimized",
            bom_name="amd",
            node_growth=None,
            tft_growth=None,
            reload=False,
        )
server=simulation.environment.node_normalized
md_code("params/default.py")

could not setup alerthandler


Input1: None

In [ ]:
md(f"""
## Simulation Arguments 

- nr of nodes at start: {simulation.nodebatches[0].nrnodes}
- nr of months left (avg out) for farming of startingnodes: {simulation.nodebatches[0].months_left}
- nr of TFT already farmed: { j.data.types.numeric.clean(simulation.nodebatches[0].tft_farmed_before_simulation)} 

""")

## Device Details

In [ ]:

md(f"""
### Device Details

- nr of TB netto usable = {round(server.su,1)} TB
- nr of cpu cores = {round(server.cru,1)} Cores
- power used = {server.power} watt
- performance in passmark = {round(server.cu_passmark*server.cu)}

""")

## Growth per month

In [ ]:
simulation.graph_nr_nodes()

### Number of racks per month


In [ ]:
x=np.arange(0,60)
cells= [int(i/44) for i in simulation.sheet.rows.rackspace_u.cells[0:60]]
fig = go.FigureWidget(data=go.Scatter(x=x, y=cells))
fig.update_layout(title = 'Racks Occupied per Month',showlegend=False)
# fig

In [ ]:
cells= [int(i) for i in simulation.sheet.rows.power_kw.cells[0:60]]
fig = go.FigureWidget(data=go.Scatter(x=x, y=cells))
fig.update_layout(title = 'Kwatt/h per Month used.',showlegend=False)
fig

## TFT Growth

In [ ]:
names=["farmed","cultivated","sold","burned","farmer_income"]
def do(end):
    fig = go.FigureWidget()
    title = 'TFT Farmed, Cultivated, Sold, Burned per month, for batch month:%s.'%end
    xx=[i for i in range(1,end+1)]
    for i in range(5):
        name=names[i]
        cells=simulation.rows["tft_%s"%name].cells[1:end+1]
        fig.add_trace(go.Scatter(x=xx, y=cells,name=name))
    fig.update_layout(title = title,showlegend=True)
    return fig


In [ ]:
do(12)

In [ ]:
do(30)

In [ ]:
do(60)

In [ ]:
fig1, fig2, fig3, fig4, fig5, fig6, fig7,fig8=simulation.graph_tft_simulation(show=False);

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig8

In [ ]:
fig3

In [ ]:
fig4


In [ ]:
fig5

In [ ]:
fig6

In [ ]:
fig7

## Cumulative TOKENS CREATED / BURNED / CULTIVATED / SOLD (in millions)


In [ ]:
m=1000000
simulation.rows.tft_sold.modify_invert()
simulation.rows.tft_burned.modify_invert()
rf=[int(i/m) for i in simulation.rows.tft_farmed.aggregate("Y",aggregate_type="SUM")]
rb=[int(i/m) for i in simulation.rows.tft_burned.aggregate("Y",aggregate_type="SUM")]
rs=[int(i/m) for i in simulation.rows.tft_sold.aggregate("Y",aggregate_type="SUM")]
rc=[int(i/m) for i in simulation.rows.tft_cultivated.aggregate("Y",aggregate_type="SUM")]

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

labels=["tft_farmed","tft_burned","tft_sold","tft_cultivated"]
for year in range(2):
    data=[rf[year],rb[year],rs[year],rc[year]]
    graph = go.Pie(labels=labels, values=data, name="Year %s"%(year+1))
    fig.add_trace(graph,1,year+1)
fig.show()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
y=1
for year in range(2,4):
    data=[rf[year],rb[year],rs[year],rc[year]]
    graph = go.Pie(labels=labels, values=data, name="Year %s"%(year+1))
    fig.add_trace(graph,1,y)
    y+=1
fig.show()
# md("### Details")



In [ ]:

fig = go.FigureWidget()
# fig.add_trace(go.Scatter(y=[2, 1, 4, 3]))
x=[1,2,3,4,5]
fig.add_trace(go.Bar(x=x,y=rf[0:5],name="Farming"))  #,text=["s sds s"]
fig.add_trace(go.Bar(x=x,y=rc[0:5],name="Cultivating"))
fig.add_trace(go.Bar(x=x,y=rs[0:5],name="Sold"))
fig.add_trace(go.Bar(x=x,y=rb[0:5],name="Burned"))
fig.update_layout(title = 'Farming / Cultivation / Burning Comparison',showlegend=True)
fig.show()

### Token price per month


In [ ]:
simulation.graph_token_price()


In [ ]:
#%qtconsole